In [1]:
%load_ext watermark
# %load_ext rpy2.ipython
import numpy as np
import pandas as pd
import matplotlib as mpl
import seaborn as sns
# import rpy2
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import beta
from scipy.stats import multinomial


import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from myst_nb import glue
from slugify import slugify

from typing import List



# from rpy2.robjects.packages import importr

import plastockconf as psc
import plastock as pstk

from plastockconf import name_zones, name_frequentation, name_situation
from plastockconf import name_substrate, name_distance, table_css_styles, table_css_styles_top

from plastock import add_table_to_page, capitalize_x_tick_labels, capitalize_x_and_y_axis_labels, capitalize_legend_components, attribute_summary

import reportclass as rc

pd.set_option('display.max_rows', 100)


In [2]:
mone = pd.read_csv('data/end_pipe/macro_data_msquared.csv')
mone["date"] = pd.to_datetime(mone['date'], format='mixed', dayfirst=True)
mone["date"] = mone["date"].dt.strftime('%Y-%m-%d')
mone["échantillon"] = list(zip(mone.Plage, mone["date"]))
mtwo = pd.read_csv('data/end_pipe/macro_current.csv')

# these needed to be removed
# not_these = [
#     'G9492',
#     'G989',
#     'G9147',
#     'G9135',
#     'G9134',
#     'G9493',
#     'G9145',
#     'G9331',
#     'G9146',
#     'G9491'
#     ]

# mtwo = mtwo[~mtwo.Code.isin(not_these)]
# mtwo.to_csv('data/end_pipe/macro_current.csv', index=False)
mtwo["Date"] = pd.to_datetime(mtwo.Date, format='mixed', dayfirst=True)
mtwo["Date"] = mtwo["Date"].dt.strftime('%Y-%m-%d')


new_names = {'Substrat':'substrat', 'Date':'date', 'Code':'code', 'Aire':'area', 'Quantité':'quantité'}
columns = ['échantillon', 'Plage', 'date', 'area', 'quantité', 'code']

t1 = mone[~mone.isna()][columns]

In [3]:
t1.head()

,échantillon,Plage,date,area,quantité,code
0,"(Amphion, 2022-02-01)",Amphion,2022-02-01,440,0,G159
1,"(Amphion, 2022-02-01)",Amphion,2022-02-01,440,0,G10
2,"(Amphion, 2022-02-01)",Amphion,2022-02-01,440,0,G103
3,"(Amphion, 2022-02-01)",Amphion,2022-02-01,440,0,G106
4,"(Amphion, 2022-02-01)",Amphion,2022-02-01,440,0,G117


In [4]:
mtwo['échantillon'] = list(zip(mtwo.Plage, mtwo.Date))
mtwo.rename(columns=new_names, inplace=True)
t2 = mtwo[columns]

In [5]:
t2.head()

,échantillon,Plage,date,area,quantité,code
0,"(Amphion, 2022-02-01)",Amphion,2022-02-01,98,2,G24
1,"(Amphion, 2022-02-01)",Amphion,2022-02-01,342,42,G24
2,"(Amphion, 2022-05-03)",Amphion,2022-05-03,342,5,G24
3,"(Amphion, 2022-07-19)",Amphion,2022-07-19,98,5,G24
4,"(Amphion, 2022-07-19)",Amphion,2022-07-19,342,11,G24


In [6]:
t2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2147 entries, 0 to 2146
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   échantillon  2147 non-null   object
 1   Plage        2147 non-null   object
 2   date         2147 non-null   object
 3   area         2147 non-null   int64 
 4   quantité     2147 non-null   int64 
 5   code         2147 non-null   object
dtypes: int64(2), object(4)
memory usage: 100.8+ KB


In [7]:
t1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7154 entries, 0 to 7153
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   échantillon  7154 non-null   object
 1   Plage        7154 non-null   object
 2   date         7154 non-null   object
 3   area         7154 non-null   int64 
 4   quantité     7154 non-null   int64 
 5   code         7154 non-null   object
dtypes: int64(2), object(4)
memory usage: 335.5+ KB


In [8]:
t1['quantité'].sum()

28107

In [9]:
t2['quantité'].sum()

28107

In [10]:
t2["échantillon"].nunique()

98

In [11]:
t1["échantillon"].nunique()

98

## Unknown codes

In [12]:
mone_codes = mone.code.unique()
mtwo_codes = mtwo.code.unique()
gfrag_gfoam_gcaps = ['G21', 'G23', 'G24', 'G78', 'G79', 'G80', 'G81', 'G82',]
diff_mtwo_mone = set(mtwo_codes).difference(set(mone_codes))

codes_in_mtwo_not_in_mone = [x for x in diff_mtwo_mone if x not in gfrag_gfoam_gcaps]
codes_in_mtwo_not_in_mone

[]

In [13]:
# codes in mtwo not in mone:
mt_wtf = mtwo[mtwo.code.isin(codes_in_mtwo_not_in_mone)]
mt_wtf['quantité'].sum()

0

## Find the errors

C'est possible que il'n'y a pas de distinction entre les deux zones pour revernges le 13 août. 

In [14]:
t2 = t2[~t2.code.isin(codes_in_mtwo_not_in_mone)]

t1_dt = t1.groupby("échantillon")[["quantité"]].sum()
t2_dt = t2.groupby("échantillon")[["quantité"]].sum()

t1_dt['s'] = [str(x) for x in t1_dt.index]
t2_dt['s'] = [str(x) for x in t2_dt.index]

t1_dt.set_index('s', inplace=True)
t2_dt.set_index('s', inplace=True)

### difference

In [15]:
print(f'Il y a {t2_dt["quantité"].sum() - t1_dt["quantité"].sum()} plus d\'objets en t2')

Il y a 0 plus d'objets en t2


In [16]:
t1_dt['t2_q'] = t1_dt.index.map(lambda x: t2_dt.loc[x, 'quantité'])
t1_dt['diff'] = t1_dt['quantité'] - t1_dt.t2_q

t1_dt['diff'].values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [17]:
event_total = mtwo.groupby(["échantillon", "Plage", "date", "Position", "area"], as_index=False)["quantité"].sum()
event_total['pcs/m²'] = event_total["quantité"]/event_total.area

### compare surface areas

In [18]:
event_total['pcs/m²'].describe()

count    185.000000
mean       0.758360
std        1.032423
min        0.005263
25%        0.179104
50%        0.377358
75%        0.944444
max        7.585859
Name: pcs/m², dtype: float64

In [19]:
mtest = mone.groupby(["échantillon", "Plage", "date", "area"], as_index=False)["quantité"].sum()
mtest["pcs/m²"] = mtest["quantité"]/mtest.area
mtest["pcs/m²"].describe()

count    98.000000
mean      0.659221
std       0.660652
min       0.025974
25%       0.204404
50%       0.469382
75%       0.845195
max       3.554572
Name: pcs/m², dtype: float64

### Extrapolated values

In [20]:
from scipy.stats import multinomial
from scipy.stats import norm

### Microplastics 1'266 712 044 - 1'349 710 956

In [57]:
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

# bayes
# given quantiles and their corresponding values
quantiles = np.array([0.01, 0.25, 0.50, 0.75, 0.99])
quantile_values = np.array([1, 4, 46, 110, 248, 460, 3000])

# min and max values are the first and last in quantile_values
min_value = quantile_values[0]
max_value = quantile_values[-1]
sampled
# linearly interpolate a CDF based on the quantiles
cdf = interp1d(quantiles, quantile_values[1:-1], bounds_error=False, fill_value=(min_value, max_value), kind='linear')
sampled = []
samples = 100*68399
# sample rom the CDF's range and invert the CDF to get samples
for _ in range(50):
    uniform_samples = np.random.uniform(0, 1, size=samples)
    samples_c = cdf(uniform_samples)
    sampled.append(samples_c.sum())

In [58]:
print(np.mean(sampled), np.median(sampled), np.quantile(sampled, [.5, .25, .5, .75, .95]))

1266712044.614237 1266732029.1250787 [1.26673203e+09 1.26618956e+09 1.26673203e+09 1.26727032e+09
 1.26788735e+09]


In [59]:
# random forest
quantile_values = np.array([1, 54, 113, 137, 149, 520, 3000])
min_value = quantile_values[0]
max_value = quantile_values[-1]
sampled = []
# Linearly interpolate a CDF based on the quantiles
cdf = interp1d(quantiles, quantile_values[1:-1], bounds_error=False, fill_value=(min_value, max_value), kind='linear')

# Sample uniformly from the CDF's range and invert the CDF to get samples
for _ in range(50):
    uniform_samples = np.random.uniform(0, 1, size=samples)
    samples_c = cdf(uniform_samples)
    sampled.append(samples_c.sum())

In [60]:
print(np.mean(sampled), np.median(sampled), np.quantile(sampled, [.5, .25, .5, .75, .95]))

1349710956.3528557 1349655583.7330441 [1.34965558e+09 1.34927715e+09 1.34965558e+09 1.35013224e+09
 1.35091905e+09]


### Macroplastics total pieces 47,100 - 70,651

In [61]:
# bayes
quantile_values = np.array([.01, .03, .14, .34, 1.06, 4.58, 10])
min_value = quantile_values[0]
max_value = quantile_values[-1]
sampled = []
# Linearly interpolate a CDF based on the quantiles
cdf = interp1d(quantiles, quantile_values[1:-1], bounds_error=False, fill_value=(min_value, max_value), kind='linear')
samples = 68399
# Sample uniformly from the CDF's range and invert the CDF to get samples
for _ in range(50):
    uniform_samples = np.random.uniform(0, 1, size=samples)
    samples_c = cdf(uniform_samples)
    sampled.append(samples_c.sum())

In [62]:
print(np.mean(sampled), np.median(sampled), np.quantile(sampled, [.5, .25, .5, .75, .95]))

70651.34697759344 70672.78455114464 [70672.78455114 70411.29306953 70672.78455114 70937.4276092
 71374.13307123]


In [63]:
# random forest
quantile_values = np.array([.01, .10, .24, .47, .6, 2.12, 10])
min_value = quantile_values[0]
max_value = quantile_values[-1]
sampled = []
# Linearly interpolate a CDF based on the quantiles
cdf = interp1d(quantiles, quantile_values[1:-1], bounds_error=False, fill_value=(min_value, max_value), kind='linear')
samples = 68399
# Sample uniformly from the CDF's range and invert the CDF to get samples
for _ in range(50):
    uniform_samples = np.random.uniform(0, 1, size=samples)
    samples_c = cdf(uniform_samples)
    sampled.append(samples_c.sum())

In [64]:
print(np.mean(sampled), np.median(sampled), np.quantile(sampled, [.5, .25, .5, .75, .95]))

47197.04576489083 47216.12036251611 [47216.12036252 47018.03671376 47216.12036252 47377.82274265
 47645.53493148]
